In [24]:
import logging
from pathlib import Path

import sys

from product_bundle.utils import load_product_df

sys.path.append("/home/xbankov/product_bundle/src/")

import pandas as pd
import networkx as nx
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from product_bundle.dataset import read_dataset, split
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

data_dir = Path("../data")

logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
data = read_dataset(data_dir)
product_df = load_product_df(data_dir)

In [36]:
import networkx as nx

G = nx.DiGraph()

In [37]:
products = product_df["StockCode"].unique()

# Add products as nodes to the graph
for product in products:
    G.add_node(product)

relationships = data.groupby(["InvoiceNo"])["StockCode"].apply(tuple).to_list()

# Add edges to represent relationships
G.add_edges_from(relationships)

# You can visualize the graph if neededd
import matplotlib.pyplot as plt

pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_size=1000, node_color="lightblue")
plt.show()

In [ ]:
# Define node features (you can customize this based on your data)
# Here, we use the total number of products in each transaction as a feature
node_features = torch.tensor([len(G.nodes[i]['products']) for i in G.nodes()], dtype=torch.float32).unsqueeze(1)

In [ ]:
node_features.shape

torch.Size([1516, 1])

In [ ]:
# Split the data into train and test nodes
nodes = list(G.nodes())
train_nodes, test_nodes = train_test_split(nodes, train_size=0.8, random_state=42)

# Create adjacency matrix
adj_matrix = nx.to_numpy_matrix(G, nodelist=nodes)

# Define a simple GNN model
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.gc1 = nn.Linear(input_dim, hidden_dim)
        self.gc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, adj_matrix):
        x = self.relu(self.gc1(x))
        x = torch.matmul(adj_matrix, x)  # Aggregate information from connected transactions
        x = self.gc2(x)
        return x

# Convert node features to PyTorch tensor
node_features = torch.tensor(node_features, dtype=torch.float32)

# Create GNN model
input_dim = 1  # Only using the number of products as a feature
hidden_dim = 64
output_dim = 1  # We'll predict a single value (e.g., bundle score)
model = GNN(input_dim, hidden_dim, output_dim)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the GNN on the training nodes
train_indices = [nodes.index(node) for node in train_nodes]
train_features = node_features[train_indices]
train_adj_matrix = adj_matrix[train_indices][:, train_indices]

for epoch in range(50):
    optimizer.zero_grad()
    outputs = model(train_features, torch.tensor(train_adj_matrix, dtype=torch.float32))
    loss = criterion(outputs, train_features)  # Reconstruction loss (minimize the difference)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}/{50}, Loss: {loss.item()}')

# Now, you can use the trained GNN to recommend bundles for test nodes
test_indices = [nodes.index(node) for node in test_nodes]
test_features = node_features[test_indices]
test_adj_matrix = adj_matrix[test_indices][:, test_indices]

with torch.no_grad():
    test_recommendations = model(test_features, torch.tensor(test_adj_matrix, dtype=torch.float32)).numpy()


AttributeError: module 'networkx' has no attribute 'to_numpy_matrix'